In [1]:
import json
import nltk
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
from gensim.models import KeyedVectors
import os
from nltk.tokenize import word_tokenize

from src.core import file_manager as fm

2022-06-19 17:26:06.645515: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-19 17:26:06.645532: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# def read_annotated_df_with_old_embeddings(embedding_name, actor='patient', variation='without_others_intent/k100_without_sentences_higher_than_median'):
#     df = fm.read_json_of_dir(
#         fm.filename_from_data_dir(
#             f'embeddings/{embedding_name}/text_emb_{actor}.json.old'),
#         lines=True
#     )

#     file_name = fm.filename_from_data_dir(
#         f'output/{actor}/{variation}/{embedding_name}/annotated_sentences.csv'
#     )

#     df_annotated = pd.read_csv(file_name)

#     df_merged = pd.merge(df_annotated, df, on='txt', how='left')

#     return df_merged


# df_old = read_annotated_df_with_old_embeddings('glove', variation='k100')
# df = fm.read_annotated_df_with_embeddings('glove', variation='k100')
# df['embeddings'].equals(df_old['embeddings'])

# df_old = read_annotated_df_with_old_embeddings('bert_pt', variation='k100')
# df = fm.read_annotated_df_with_embeddings('bert_pt', variation='k100')
# df['embeddings'].equals(df_old['embeddings'])

## Sentences

In [3]:
df = fm.read_annotated_df_with_embeddings('glove')

In [4]:
vocab = {}

for index, row in df.iterrows():
  for index, token in enumerate(row['tokens']):
    if token not in vocab:
      vocab[token] = row['word_embeddings'][index]

n_source_words = len(vocab)

print(n_source_words)

7150


In [5]:
df['intent'].value_counts()

inform_symptoms    7120
greeting            948
request_inform      492
inform_medicine     340
Name: intent, dtype: int64

In [6]:
max_len = df['tokens'].apply(lambda x : len(x)).max()
print(max_len)
# max_len = 700 #75% do dataset pelo menos tem no maximo esse valor de tamanho

155


In [7]:
def create_embedding_matrix(vocab, n_words, emb_size):
    
    pret_embedding = {}
    embedding_matrix = np.zeros((n_words, emb_size))
    count = 0

    # Dicionário com todos os ids e palavras do embedding pré-treinado
    # for index, word in enumerate(vocab.index_to_key):
    for index, word in enumerate(vocab.keys()):
      pret_embedding[word] = index

    # Construindo a embedding_matrix do embedding pré-treinado
    for item in pret_embedding.items():
      if item[1] < n_words:
        count += 1
        embedding_vector = vocab[item[0]]
        if embedding_vector is not None:
          embedding_matrix[item[1]] = embedding_vector
      
    return embedding_matrix

## Save the vectors  in a new Matrix

In [8]:
# embedding_size = 300
embedding_size = len(df.loc[0].word_embeddings[0])
embedding_matrix = create_embedding_matrix(vocab, n_source_words, embedding_size)
embedding_matrix.shape

(7150, 300)

In [9]:
vocab_key_2_index =  {key: index for index, key in enumerate(vocab.keys())}

In [10]:
type(embedding_matrix), type(vocab), type(vocab_key_2_index)

(numpy.ndarray, dict, dict)

## Padding

In [11]:
# Função que cria o X
def create_x(tokens):
    list_x = []
    for token in tokens:
      if(token in vocab_key_2_index):
        list_x.append(vocab_key_2_index[token])
      
    return list_x


In [12]:
df['token_indexes'] = df['tokens'].apply(lambda tokens : create_x(tokens))
df.head(5)

,txt,annotated_txt,intent,embeddings,tokens,word_embeddings,token_indexes
0,Com muito medo,Com muito medo,inform_symptoms,"[[0.07444667, -0.04100433, -0.39161667, 0.1238...","[Com, muito, medo]","[[0.039064, 0.34595000000000004, 0.11547, -0.4...","[0, 1, 2]"
1,Tá certo,Tá certo,greeting,"[[-0.0368989, 0.004874997000000001, 0.14075000...","[Tá, certo]","[[-0.0089978, -0.15736, 0.57216, 0.49819, -0.2...","[3, 4]"
2,Tá bom,Tá bom,greeting,"[[0.09625109999999999, -0.21625000000000003, 0...","[Tá, bom]","[[-0.0089978, -0.15736, 0.57216, 0.49819, -0.2...","[3, 5]"
3,Boa tarde! Tenho uma idosa de 84 anos e vivi n...,Boa tarde! Tenho uma idosa de 84 anos e vivi n...,inform_symptoms,"[[0.008652593, -0.048798796000000005, -0.15833...","[Boa, tarde, !, Tenho, uma, idosa, de, 84, ano...","[[0.077468, -0.15121, -0.5625300000000001, 0.4...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1..."
4,Acabei de enviar,Acabei de enviar,inform_symptoms,"[[0.07401667, -0.06999668, -0.2369833400000000...","[Acabei, de, enviar]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[93, 12, 94]"


In [13]:
X = pad_sequences(maxlen=max_len, sequences=df['token_indexes'], value=0, padding='post', truncating='post')
Y = pd.get_dummies(df['intent']).values
(X.shape, Y.shape)

((8900, 155), (8900, 4))

## Create train and test

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42,stratify=Y)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7120, 155) (7120, 4)
(1780, 155) (1780, 4)


## LSTM Model

In [15]:
num_labels = Y.shape[1]
MAX_NB_WORDS = n_source_words
# EMBEDDING_DIM = 50
EMBEDDING_DIM = embedding_size

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1], weights=[embedding_matrix]))
# model.add(Embedding(X.shape[0], X.shape[1], input_length=X.shape[1], weights=[X]))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.1))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#Optimisation functions usually calculate the gradient i.e. the partial derivative of loss function with respect to weights, 
#and the weights are modified in the opposite direction of the calculated gradient.

2022-06-19 17:26:16.990687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-19 17:26:16.991034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-19 17:26:16.991066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-19 17:26:16.991092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-19 17:26:16.991120: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 155, 300)          2145000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              186880    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 4)                 516       
                                                                 
Total params: 2,332,396
Trainable params: 2,332,396
Non-trainable params: 0
_________________________________________________________________


In [17]:
epochs = 20
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/20
101/101 [==============================] - 10s 87ms/step - loss: 0.3964 - accuracy: 0.8663 - val_loss: 0.1653 - val_accuracy: 0.9522
Epoch 2/20
101/101 [==============================] - 8s 83ms/step - loss: 0.1094 - accuracy: 0.9677 - val_loss: 0.0980 - val_accuracy: 0.9691
Epoch 3/20
101/101 [==============================] - 8s 83ms/step - loss: 0.0627 - accuracy: 0.9803 - val_loss: 0.0845 - val_accuracy: 0.9747
Epoch 4/20
101/101 [==============================] - 8s 83ms/step - loss: 0.0356 - accuracy: 0.9885 - val_loss: 0.0852 - val_accuracy: 0.9747
Epoch 5/20
101/101 [==============================] - 8s 83ms/step - loss: 0.0223 - accuracy: 0.9939 - val_loss: 0.1218 - val_accuracy: 0.9705
Epoch 6/20
101/101 [==============================] - 9s 84ms/step - loss: 0.0154 - accuracy: 0.9966 - val_loss: 0.1151 - val_accuracy: 0.9705


## Save model the model and vocabullary

In [18]:
working_dir = fm.filename_from_data_dir(f'output/lstm_models/patient/glove')

In [19]:
print('saving the model')
model.save(f'{working_dir}/model.h5')

saving the model


In [20]:
print('saving the vocabullary')

file = open(f'{working_dir}/vocabullary.json',"w")

file.write(json.dumps(vocab_key_2_index))

file.close()

saving the vocabullary


In [21]:
print('saving the metadata')

file = open(f'{working_dir}/metadata.json',"w")

intents = pd.get_dummies(df['intent']).columns.tolist()

metadata = {
  'intents': intents,
  'vector_length': str(max_len)
}

file.write(json.dumps(metadata))

file.close()

saving the metadata


## Evaluate the model

In [22]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

56/56 [==============================] - 1s 14ms/step - loss: 0.1436 - accuracy: 0.9618
Test set
  Loss: 0.144
  Accuracy: 0.962
